In [1]:
# Imports
import pandas as pd
import requests
import time

from bs4 import BeautifulSoup

In [2]:
# Creating empty list for all links
links_list = [] 

# Looping through all songs pages to create master list of lyrics links
# At time of scraping there are 22 pages of songs
for n in range(1, 23):
    lyrics_links_res = requests.get(f'http://www.shantynet.com/category/lyrics/page/{n}/')
    print(f'{n} {lyrics_links_res.status_code}')
    
    lyrics_links_soup = BeautifulSoup(lyrics_links_res.content, 'lxml')
    lyrics_links_links = lyrics_links_soup.find_all('a', href=True, title=True, rel=True)
    lyrics_links_links = [link['href'] for link in lyrics_links_links]
    links_list = links_list + lyrics_links_links

1 200
2 200
3 200
4 200
5 200
6 200
7 200
8 200
9 200
10 200
11 200
12 200
13 200
14 200
15 200
16 200
17 200
18 200
19 200
20 200
21 200
22 200


In [30]:
# Creating empty dictionary to store songs and lyrics
shanties_dict = {}

In [ ]:
# Looping through links list to extract lyrics
for link in links_list:
    lyrics_res = requests.get(link)
    # Putting in a small sleep to limit requests on server to under 100 per minutes
    time.sleep(.75)
    
    # Printing status codes as check for scraping progress
    print(f'{lyrics_res.status_code}')
    
    lyrics_soup = BeautifulSoup(lyrics_res.content, 'lxml')

    # Pulling song title
    title = BeautifulSoup(lyrics_res.text).title.text
    title = title.replace('Shanty Net', '')

    # Checking lyric format type. If html is written with few 'p' tags will do if otherwise else
    if len(lyrics_soup.find_all('p')) < 15:
        print('div tag song')
        
        # Class string looks to be unique to lyrics
        lyrics_raw = lyrics_soup.find_all('div', class_='_UZe kno-fb-ctx')
        lyrics_raw = [x.text for x in lyrics_raw]
        lyrics = ''
        lyrics = lyrics.join(lyrics_raw)
        shanties_dict.update({title : lyrics})
    
    else:   
        lyrics_raw = lyrics_soup.find_all('p')
        
        # Deleting not lyrics 'p' tags
        del lyrics_raw[-9:]

        # List of words to check through at begining of song
        check_list = ['written', 'music', 'lyrics' 'www.']

        # If one of the words above is found it will delete the authorship attribution line from lyrics
        for check in check_list:
            if check in lyrics_raw[0].text.lower():
                del lyrics_raw[0]
                print('Deleted a writer line')
                break
                
        lyrics_raw = [x.text for x in lyrics_raw]
        lyrics = ''
        lyrics = lyrics.join(lyrics_raw)
        shanties_dict.update({title : lyrics})

In [32]:
# Creating data frame for songs and lyrics
shanty_net_songs = pd.DataFrame(shanties_dict.items(), columns=['title', 'lyrics'])

In [8]:
# Writing to csv for use in other scripts
shanty_net_songs.to_csv('../data/shanty_net_songs.csv')